## Version 3
Okay we need to refactor this whole thing (version 02), to be more maintainable, and prepared for putting into production.
Also the devops guys say they are now enforcing pull requests with code quality checking so our code needs to be less "smelly" or the pull request won't be accepted. Installing Ruff can help with that apparently.  
Also DONT COMMIT DIRECTLY TO MAIN!

Add autoreload since we are now modifying scripts in other folders.

In [1]:
%load_ext autoreload
%autoreload 2

Getting the data again.

In [2]:

import sys
import os

sys.path.append(os.path.abspath(".."))

from ARISA_DSML.preproc import get_raw_data
from ARISA_DSML.config import RAW_DATA_DIR, target, categorical

2025-03-24 21:39:52.673 | INFO     | ARISA_DSML.config:<module>:12 - PROJ_ROOT path is: C:\Users\Kinga\OneDrive - Wojskowa Akademia Techniczna\Podyplomowe\ARISA-ML-OPS\ARISA-MLOps


In [3]:

import os
from pathlib import Path
container_check = os.getenv("iscontainer")
if container_check=="y":
    config_dir = Path("/home/vscode/.config/kaggle")
    config_dir.mkdir(parents=True, exist_ok=True)

    with open(config_dir / "kaggle.json", "w") as dst:
        with open("../kaggle.json", "r") as src:
            dst.write(src.read())

In [4]:
get_raw_data()
!dir "{RAW_DATA_DIR}"

2025-03-24 21:39:53.701 | INFO     | ARISA_DSML.preproc:get_raw_data:22 - RAW_DATA_DIR is: C:\Users\Kinga\OneDrive - Wojskowa Akademia Techniczna\Podyplomowe\ARISA-ML-OPS\ARISA-MLOps\data\raw


Dataset URL: https://www.kaggle.com/datasets/wesleyhowe/titanic-labelled-test-set
 Volume in drive C is OS
 Volume Serial Number is 883F-45C6

 Directory of C:\Users\Kinga\OneDrive - Wojskowa Akademia Techniczna\Podyplomowe\ARISA-ML-OPS\ARISA-MLOps\data\raw

24.03.2025  21:39    <DIR>          .
24.03.2025  19:15    <DIR>          ..
24.03.2025  19:14                 0 .gitkeep
24.03.2025  21:39             3�258 gender_submission.csv
24.03.2025  21:39            28�629 test.csv
24.03.2025  21:39            30�234 test_augmented.csv
24.03.2025  21:39            61�194 train.csv
               5 File(s)        123�315 bytes
               2 Dir(s)  174�850�740�224 bytes free


Load preproc from preproc.py and run on train data:

In [5]:
from ARISA_DSML.preproc import preprocess_df

train_path = preprocess_df(RAW_DATA_DIR / "train.csv")

In [6]:
train_path

WindowsPath('C:/Users/Kinga/OneDrive - Wojskowa Akademia Techniczna/Podyplomowe/ARISA-ML-OPS/ARISA-MLOps/data/processed/train.csv')

In [7]:
import pandas as pd
df_train = pd.read_csv(train_path)
df_train["Title"].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    object 
 9   Title        891 non-null    object 
 10  Deck         891 non-null    object 
 11  CabinNumber  891 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 83.7+ KB


Split into X and y, get categorical indices:

In [9]:
y_train = df_train.pop(target)
X_train = df_train

categorical_indices = [X_train.columns.get_loc(col) for col in categorical if col in X_train.columns]
categorical_indices

[1, 2, 7, 9, 8]

And run the hyperparameter optimization (if ):

In [10]:
from ARISA_DSML.train import run_hyperopt

best_params_path = run_hyperopt(X_train, y_train, categorical_indices)

2025-03-24 21:40:44.385 | INFO     | ARISA_DSML.train:run_hyperopt:72 - Best Parameters: {'depth': 4, 'learning_rate': 0.18957539727921802, 'iterations': 120, 'l2_leaf_reg': 1.0864007479797192e-05, 'bagging_temperature': 0.737390574446251, 'random_strength': 9.87941178103755}


Do cross validation again:

In [11]:
import joblib
from ARISA_DSML.train import train_cv
params = joblib.load(best_params_path)
display(params)
n_folds = 5
cv_output_path = train_cv(X_train, y_train, categorical_indices, params, n=n_folds)

{'depth': 4,
 'learning_rate': 0.18957539727921802,
 'iterations': 120,
 'l2_leaf_reg': 1.0864007479797192e-05,
 'bagging_temperature': 0.737390574446251,
 'random_strength': 9.87941178103755}

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]
0:	learn: 0.7185841	test: 0.7913669	best: 0.7913669 (0)	total: 172ms	remaining: 20.5s
1:	learn: 0.7208481	test: 0.7913669	best: 0.7913669 (0)	total: 189ms	remaining: 11.2s
2:	learn: 0.7186933	test: 0.7794118	best: 0.7913669 (0)	total: 203ms	remaining: 7.9s
3:	learn: 0.7285714	test: 0.7971014	best: 0.7971014 (3)	total: 218ms	remaining: 6.31s
4:	learn: 0.7302752	test: 0.7851852	best: 0.7971014 (3)	total: 226ms	remaining: 5.19s
5:	learn: 0.7279412	test: 0.7851852	best: 0.7971014 (3)	total: 242ms	remaining: 4.6s
6:	learn: 0.7293233	test: 0.7819549	best: 0.7971014 (3)	total: 255ms	remaining: 4.11s
7:	learn: 0.7316176	test: 0.7851852	best: 0.7971014 (3)	total: 272ms	remaining: 3.81s
8:	learn: 0.7339450	test: 0.7851852	best: 0.7971014 (3)	total: 284ms	remaining: 3.51s
9:	learn: 0.7422303	test: 0.7851852	best: 0.7971014 (3)	total: 301ms	remaining: 3.31s
10:	learn: 0.7384045	test: 0.7851852	best: 0.7971014 (3)	total: 317ms	remaining: 3.14s
11:	learn: 0.7397770	test: 0.785

And get the performance plots:

In [ ]:
from ARISA_DSML.train import plot_error_scatter
cv_results = pd.read_csv(cv_output_path)
plot_error_scatter(
    df_plot=cv_results,
    name="Mean F1 Score",
    title="Cross-Validation (N=5) Mean F1 score with Error Bands",
    xtitle="Training Steps",
    ytitle="Performance Score",
    yaxis_range=[0.5, 1],
)

In [ ]:
plot_error_scatter(
    df_plot=cv_results,
    x="iterations",
    y="test-Logloss-mean",
    err="test-Logloss-std",
    name="Mean logloss",
    title="Cross-Validation (N=5) Mean Logloss with Error Bands",
    xtitle="Training Steps",
    ytitle="Logloss",
)


Fit model on full dataset:

In [ ]:
from ARISA_DSML.train import train
model_path, model_params_path = train(X_train, y_train, categorical_indices, params)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7114504	total: 14.2ms	remaining: 1.68s
50:	learn: 0.7914110	total: 639ms	remaining: 865ms
100:	learn: 0.8558282	total: 1.39s	remaining: 261ms
119:	learn: 0.8646154	total: 1.68s	remaining: 0us


TypeError: 'NoneType' object is not subscriptable

And now for the test set, because of our refactor, we don't need to copy paste anymore:

In [ ]:
dataset_test_path = RAW_DATA_DIR / "test.csv"
preprocced_test_path = preprocess_df(dataset_test_path)
df_test = pd.read_csv(preprocced_test_path)
df_test["Title"].unique()

df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,CabinNumber
0,892,3,male,34.5,0,0,7.8292,Q,Mr,N,47
1,893,3,female,47.0,1,0,7.0000,S,Mrs,N,47
2,894,2,male,62.0,0,0,9.6875,Q,Mr,N,47
3,895,3,male,27.0,0,0,8.6625,S,Mr,N,47
4,896,3,female,22.0,1,1,12.2875,S,Mrs,N,47


In [ ]:
df_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,CabinNumber
0,1,3,male,22.0,1,0,7.2500,S,Mr,N,50
1,2,1,female,38.0,1,0,71.2833,C,Mrs,C,85
2,3,3,female,26.0,0,0,7.9250,S,Miss,N,50
3,4,1,female,35.0,1,0,53.1000,S,Mrs,C,123
4,5,3,male,35.0,0,0,8.0500,S,Mr,N,50


Get predictions and shap plot:

In [ ]:
from ARISA_DSML.predict import predict
from catboost import CatBoostClassifier

params = joblib.load(model_params_path)
display(params)
from_file = CatBoostClassifier()
model = from_file.load_model(str(model_path))
preds_path = predict(model, df_test, params)


NameError: name 'model_params_path' is not defined

And that's it for the refactor.  
As can be seen from the notebook, the goal is to have as little code as possible in the actual notebook cells,  
which makes it much easier to maintain and, as we will see shortly, productionalize.